# Prompt Walkthrough

An overview of the different types of prompts in LangChain and how to use them

### Basic Prompt

The most simple type of prompt - a string template that takes any number of input variables. The template should be formatted as a Python f-string.

In [1]:
from langchain.prompts import Prompt

In [2]:
# An example prompt with no input variables
no_input_prompt = Prompt(input_variables=[], template="Tell me a joke.")
no_input_prompt.format()

'Tell me a joke.'

In [3]:
# An example prompt with one input variable
no_input_prompt = Prompt(input_variables=["adjective"], template="Tell me a {adjective} joke.")
no_input_prompt.format(adjective="funny")

'Tell me a funny joke.'

In [4]:
# An example prompt with multiple input variables
no_input_prompt = Prompt(input_variables=["adjective", "content"], template="Tell me a {adjective} joke about {content}.")
no_input_prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

### Examples
Examples are datapoints that can be used to show the model how to produce results. They can be either strings, or dictionaries that are then turned into strings by an example prompt itself.

In [5]:
string_examples = ["Input: happy\nOutput: sad", "Input: tall\nOutput: short"]
dict_examples = [{"input": "happy", "output": "sad"}, {"input": "tall", "output": "short"}]
example_prompt = Prompt(input_variables=["input","output"], template="Input: {input}\nOutput: {output}")

### Simple Prompt with examples

We can then use these examples to construct prompts.

In [11]:
prompt_from_string_examples = Prompt.from_examples(
    string_examples, 
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:", 
    input_variables=["adjective"],
)
print(prompt_from_string_examples.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: big
Output:


In [12]:
prompt_from_string_examples = Prompt.from_structured_examples(
    dict_examples,
    example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:", 
    input_variables=["adjective"],
)
print(prompt_from_string_examples.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: big
Output:


### Dynamic Prompt

We also do more clever things with prompts - for example, only select a certain number of examples in order to limit the size of the text passed in. This will vary with the input text size.

In [13]:
from langchain.prompts import DynamicPrompt

In [17]:
dynamic_prompt = DynamicPrompt.from_structured_examples(
    dict_examples,
    example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:", 
    input_variables=["adjective"],
    max_length=20,
)

In [19]:
# An example with small input, so it selects both examples.
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: big
Output:


In [20]:
# An example with long input, so it selects only one example.
print(dynamic_prompt.format(adjective="big and huge and massive"))

Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive
Output:


# Optimized Prompt

Besides selecting a variable number of examples to show, we can also select examples that most closely match the user input. This is done by creating embeddings of the user input and comparing it embeddings of the examples.

In [23]:
from langchain.prompts.optimized import OptimizedPrompt
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [25]:
optimized_prompt = OptimizedPrompt.from_structured_examples(
    dict_examples,
    example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:", 
    input_variables=["adjective"],
    embeddings=OpenAIEmbeddings(),
    vectorstore_cls=FAISS
)

In [28]:
# Input is a feeling, so should select the happy/sad example
print(optimized_prompt.format(adjective="worried", k=1))

Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:


In [29]:
# Input is a measurment, so should select the tall/short example
print(optimized_prompt.format(adjective="fat", k=1))

Give the antonym of every input

Input: tall
Output: short

Input: fat
Output:
